In [1]:
import pandas as pd
import numpy as np
from utils import *


In [2]:
#pip install sentence-transformers

Load the datasets - Mentor/Mentee/Circle

In [3]:
Mentee_df = pd.read_excel('../data/Mentee-obfuscated-dataset-1.xlsx', engine='openpyxl')

In [4]:
Mentee_df.columns

Index(['Mentee_ID', 'Mentee_First_Name', 'Mentee_Last_Name', 'Mentee_Email',
       'Mentee_Title', 'Mentee_GradeLevel', 'Mentee_Organization',
       'Mentee_Department', 'Mentee_BusinessUnit', 'Mentee_TimeZone',
       'Mentee_Circle', 'Mentee_Goal', 'Mentee_Interest', 'Mentee_CLL',
       'Mentor_Interestcount', 'Mentee_Eligibility', 'Mentee_wiads_past'],
      dtype='object')

In [5]:
Mentor_df = pd.read_excel('../data/Mentor-obfuscated-dataset-1.xlsx', engine='openpyxl')

In [6]:
Mentor_df.columns

Index(['Mentor_ID', 'Mentor_FirstName', 'Mentor_LastName', 'Mentor_Email',
       'Mentor_JobTitle ', 'Mentor_GradeLevel', 'Mentor_Organization',
       'Mentor_Department', 'Mentor_BusinessUnit', 'Mentor_TimeZone',
       'Mentor_Circle', 'Mentor_Interest', 'Mentor_CLL', 'Mentor_Goal',
       'Mentor_Interestcount', 'Mentor_Eligibility', 'Mentor_wiads_past ',
       'Mentor_mentor_past'],
      dtype='object')

In [7]:
Circle_df = pd.read_excel('../data/CircleResult.xlsx', engine='openpyxl')

In [8]:
#Circle_df.head()

In [9]:
list(Circle_df.columns)

['Circle',
 'Mentee_ID',
 'Mentee_First_Name',
 'Mentee_Last_Name',
 'Mentee_Email',
 'Mentee_Title',
 'Mentee_GradeLevel',
 'Mentee_Organization',
 'Mentee_Department',
 'Mentee_BusinessUnit',
 'Mentee_TimeZone',
 'Mentee_Circle',
 'Mentee_Goal',
 'Mentee_Interest',
 'Mentee_CLL',
 'Mentor_ID',
 'Mentor_FirstName',
 'Mentor_LastName',
 'Mentor_Email',
 'Mentor_JobTitle\xa0',
 'Mentor_GradeLevel',
 'Mentor_Organization',
 'Mentor_Department',
 'Mentor_BusinessUnit',
 'Mentor_TimeZone',
 'Mentor_Circle',
 'Mentor_interest',
 'Mentor_CLL',
 'Mentor_Goal']

Select only 3 columns

In [10]:
Circle_basic_df = Circle_df[['Circle','Mentor_ID','Mentee_ID']]

In [11]:
#Circle_basic_df.head()
n = Circle_df.Mentee_Email.nunique()
  
print("No.of.unique values ",n)

No.of.unique values  16


In [12]:
Cicle_basic_sorted_df = Circle_basic_df.sort_values(by=['Circle','Mentor_ID','Mentee_ID'], inplace=False, ignore_index= True)

For each circle row, get the mentor and mentee information and create a detail dataframe

• Organization/Department - Mentor and Mentee should not be in same Department. 
• Time Zones – Fit circles into similar time zones. Find difference of timezone between mentor and mentee
• Grade Levels/Mentor Circle Levels – Anyone Grade level less than 28 should not be considered a mentor.
• CLL Sort – Mentors strengths match the Mentees growth areas.
• Areas of interest – Maximum match on areas of interest within a circle.
• Circle size –The number of mentees a mentor has. The circles should be close in size.
• Mentor grade error if mentor grade < 28




In [13]:
Result_Dictionary={}
Result_DF = pd.DataFrame()

In [14]:
Cicle_basic_sorted_dfdict = Cicle_basic_sorted_df.to_dict('records')
for  circle_row in Cicle_basic_sorted_dfdict:
    mentorId = circle_row['Mentor_ID']
    menteeId = circle_row['Mentee_ID']
    if (np.isnan(mentorId) or mentorId ==' ' or np.isnan(menteeId) or menteeId ==' '):
        continue
    else:        
        mentor_row = Mentor_df[Mentor_df['Mentor_ID'] ==circle_row['Mentor_ID'] ]
        mentee_row = Mentee_df[Mentee_df['Mentee_ID'] ==circle_row['Mentee_ID'] ]

        print(circle_row)
        result = validate_mentormentee(mentor_row, mentee_row, circle_row)
        result['Mentor_Email'] = mentor_row['Mentor_Email'].iloc[0]
        result['Mentee_Email'] = mentee_row['Mentee_Email'].iloc[0]
        Result_DF = Result_DF.append(result, ignore_index=True)
    

{'Circle': 1, 'Mentor_ID': 1.0, 'Mentee_ID': 4}
{'Circle': 1, 'Mentor_ID': 1.0, 'Mentee_ID': 5}
{'Circle': 1, 'Mentor_ID': 1.0, 'Mentee_ID': 6}
{'Circle': 1, 'Mentor_ID': 1.0, 'Mentee_ID': 7}
{'Circle': 1, 'Mentor_ID': 1.0, 'Mentee_ID': 8}
{'Circle': 1, 'Mentor_ID': 1.0, 'Mentee_ID': 9}
{'Circle': 1, 'Mentor_ID': 1.0, 'Mentee_ID': 15}
{'Circle': 1, 'Mentor_ID': 1.0, 'Mentee_ID': 16}
{'Circle': 1, 'Mentor_ID': 1.0, 'Mentee_ID': 17}
{'Circle': 1, 'Mentor_ID': 10.0, 'Mentee_ID': 3}
{'Circle': 2, 'Mentor_ID': 10.0, 'Mentee_ID': 10}
{'Circle': 2, 'Mentor_ID': 10.0, 'Mentee_ID': 11}
{'Circle': 2, 'Mentor_ID': 10.0, 'Mentee_ID': 12}
{'Circle': 2, 'Mentor_ID': 10.0, 'Mentee_ID': 13}
{'Circle': 2, 'Mentor_ID': 10.0, 'Mentee_ID': 14}
{'Circle': 3, 'Mentor_ID': 17.0, 'Mentee_ID': 101}


In [15]:
Result_DF.head()

,Circle,Goal_Similarity,Mentee_Circle,Mentee_Department,Mentee_Email,Mentee_Grade,Mentee_ID,Mentee_TimeZone,Mentee_TimeZoneNum,MentorMentee_CLL_Growth,...,MentorMentee_Grade_Error,Mentor_Circle,Mentor_Department,Mentor_Eligible_Error,Mentor_Email,Mentor_Grade,Mentor_ID,Mentor_Mentee_TimeZoneDist,Mentor_TimeZone,Mentor_TimeZoneNum
0,1.0,0.4033,Early Career,Ancillary,First_4.Last_4@uvw.com,27.0,4.0,EST,3.0,0.0,...,1.0,Mid Career,Specialty benefits,1.0,First_1.Last_1@corpitsvcs.com,27.0,1.0,0.0,EST,3.0
1,1.0,0.4244,Mid Career,Specialty benefits,First_5.Last_5@uvw.com,28.0,5.0,PST,6.0,1.0,...,1.0,Mid Career,Specialty benefits,1.0,First_1.Last_1@corpitsvcs.com,27.0,1.0,3.0,EST,3.0
2,1.0,0.0213,Early Career,Ancillary,First_5.Last_5@uvw.com,27.0,6.0,EST,3.0,0.0,...,1.0,Mid Career,Specialty benefits,1.0,First_1.Last_1@corpitsvcs.com,27.0,1.0,0.0,EST,3.0
3,1.0,0.4541,Early Career,Ancillary,First_6.Last_6@uvw.com,27.0,7.0,IST,0.0,2.0,...,1.0,Mid Career,Specialty benefits,1.0,First_1.Last_1@corpitsvcs.com,27.0,1.0,3.0,EST,3.0
4,1.0,0.2395,Mid Career,Government Programs,First_7.Last_7@uvw.com,27.0,8.0,CST,4.0,1.0,...,1.0,Mid Career,Specialty benefits,1.0,First_1.Last_1@corpitsvcs.com,27.0,1.0,1.0,EST,3.0


Create a summary dataframe  
• Count of members in cicle
• Min Max on interests, CLL sort
• Count of departments in cicle

In [16]:

# Groupby multiple columns & multiple aggregations
grouped = Result_DF.groupby('Circle').aggregate({'Mentor_ID':['count','nunique'],'Mentee_ID':['count','nunique'],'Mentee_TimeZoneNum':['min','max'], 'Mentor_Eligible_Error':'sum', 'MentorMentee_Grade_Error':'sum','MentorMentee_DeptError':'sum','Mentee_Department':'nunique','Goal_Similarity':['min','max'],'Mentee_Grade':['min','max'],'Mentee_TimeZoneNum':['min','max'],'Mentor_Mentee_TimeZoneDist':['min','max'],'MentorMentee_CLL_Growth':['min','max'],'MentorMentee_Circle_Error':['sum'],'MentorMentee_Common_Interests':['min','max']})
regroup_DF = grouped.reset_index()
regroup_DF.head()

Circle Mentor_ID         Mentee_ID         Mentee_TimeZoneNum       \
             count nunique     count nunique                min  max   
0    1.0        10       2        10      10                0.0  6.0   
1    2.0         5       1         5       5                3.0  6.0   
2    3.0         1       1         1       1                5.0  5.0   

  Mentor_Eligible_Error MentorMentee_Grade_Error MentorMentee_DeptError  ...  \
                    sum                      sum                    sum  ...   
0                   9.0                      8.0                    1.0  ...   
1                   0.0                      3.0                    0.0  ...   
2                   0.0                      0.0                    0.0  ...   

  Goal_Similarity Mentee_Grade       Mentor_Mentee_TimeZoneDist       \
              max          min   max                        min  max   
0          0.4821         25.0  29.0                        0.0  3.0   
1          0.3878         26.0  29.0                        0.0  2.0   
2          0.4637         26.0  26.0                        2.0  2.0   

  MentorMentee_CLL_Growth      MentorMentee_Circle_Error  \
                      min  max                       sum   
0                     0.0  2.0                       5.0   
1                     0.0  1.0                       2.0   
2                     1.0  1.0                       1.0   

  MentorMentee_Common_Interests       
                            min  max  
0                           0.0  2.0  
1                           0.0  1.0  
2                           1.0  1.0  

[3 rows x 22 columns]

Calculate Circle level Penalty and Bonus

• Organization/Department - To encourage networking the circles mentees from the same organization should be split between different circles. Mentor and Mentee should not be in same Department.
• Time Zones – Fit circles into similar time zones or within 2-hour differences. *International participants placed in eastern time zone groups.
• Grade Levels/Mentor Circle Levels – Should not have more than three grade levels between participants. Appropriate grade level grouping of mentor and mentees. Circles should not have both Early Career and Mid Career members.
Anyone Grade level less than 28 should not be considered a mentor.
• CLL Sort – Mentors strengths match the Mentees growth areas.
• Areas of interest – Maximum match on areas of interest within a circle.
• Circle size –The number of mentees a mentor has. The circles should be close in size.


In [17]:
#Set Base Score as 100
Base_Score = 100

 Create Circle summary score and explanation`

In [18]:

Mentee_Department_In_Circle_Error_Group = Result_DF.groupby(['Circle', 'Mentee_Department']).filter(lambda x: len(x) >= 2).Circle.unique()


In [19]:
Circle_Penalty, Circle_Bonus,  Circle_Summary_DF = calculateCircleLevelScore(regroup_DF, Mentee_Department_In_Circle_Error_Group)


In [20]:
Circle_Summary_DF

,Circle,Circle Size,Circle Size Penalty,Mentee Grade Max,Mentee Grade Min,Mentee Grade Span,Mentee Mentor Department Error,Mentee Mentor Department Penalty,Mentee Mentor Grade Penalty,Mentee Mentor TimeZone Difference,...,Mentor Mentee Circle Error,Mentor Mentee Circle Penalty,Mentor Mentee Goal Similarity Bonus,Mentor Mentee Interest Bonus,Mentor Mentee Min Goal Similarity,Mentor Mentee Minimum Interest Commonality,Too Many Mentees in Same Department Penalty,Unique Mentee Departments,Unique Mentors,Unique Mentors Penalty
0,1.0,10.0,0.0,29.0,25.0,4.0,1.0,-1.0,-1.0,3.0,...,5.0,-1.0,0.0,0.0,0.0213 - 0.4821,0.0 - 2.0,-1.0,8.0,2.0,-2.0
1,2.0,5.0,0.0,29.0,26.0,3.0,0.0,0.0,0.0,2.0,...,2.0,-1.0,0.0,0.0,0.1314 - 0.3878,0.0 - 1.0,0.0,5.0,1.0,0.0
2,3.0,1.0,0.0,26.0,26.0,0.0,0.0,0.0,0.0,2.0,...,1.0,-1.0,2.0,0.0,0.4637 - 0.4637,1.0 - 1.0,0.0,1.0,1.0,0.0


In [21]:
Circle_Summary_DF.columns

Index(['Circle', 'Circle Size', 'Circle Size Penalty', 'Mentee Grade Max',
       'Mentee Grade Min', 'Mentee Grade Span',
       'Mentee Mentor Department Error', 'Mentee Mentor Department Penalty',
       'Mentee Mentor Grade Penalty', 'Mentee Mentor TimeZone Difference',
       'Mentee Mentor TimeZone Penalty', 'Mentees TimeZone Difference',
       'Mentees TimeZone Penalty', 'Mentees belonging to same Department',
       'Mentor Eligibility Penalty', 'Mentor Eligible Error',
       'Mentor Mentee CLL Growth Bonus',
       'Mentor Mentee CLL Growth Commonality', 'Mentor Mentee Circle Error',
       'Mentor Mentee Circle Penalty', 'Mentor Mentee Goal Similarity Bonus',
       'Mentor Mentee Interest Bonus', 'Mentor Mentee Min Goal Similarity',
       'Mentor Mentee Minimum Interest Commonality',
       'Too Many Mentees in Same Department Penalty',
       'Unique Mentee Departments', 'Unique Mentors',
       'Unique Mentors Penalty'],
      dtype='object')

In [22]:
print("Circle score is ",Circle_Bonus, Circle_Penalty)

Circle score is  2 -11


Create high level statistics for easier evaluation of bonus and penalty points

Penalty points:
    Circle size > 10
    Mentor and mentee are in same department
    Number of mentees not matched to any mentor
Bonus Points
    Timezone 
    Grade Level distance <= 3
    Drop mentors with grade level < 28
      
      
• Max - Min circle size
• Circle Size > 10
• Number of mentees not in any circle
• Number of mentors not in any circle
• Duplicate entry for mentee

In [23]:
#Get High Level Score 
(Summary_Score, Summary_Results) = createSummary(Result_DF, regroup_DF)


In [24]:
##Find unassigned mentors, mentees
unassignedMentees = set(Mentee_df.Mentee_Email).difference(set(Result_DF.Mentee_Email))
unassignedMentors = set(Mentor_df.Mentor_Email).difference(set(Result_DF.Mentor_Email))
Summary_Results['Unassigned Mentees']=";".join(unassignedMentees)
Summary_Results['Number of Unassigned Mentees']=len(unassignedMentees)
Summary_Results['Unassigned Mentors']=";".join(unassignedMentors)
Summary_Results['Number of Unassigned Mentors']=len(unassignedMentors)

In [25]:
#Add scores to Summary Results
Final_Score = Base_Score + Circle_Bonus + Circle_Penalty + Summary_Score
Summary_Results['Base Score'] = Base_Score
Summary_Results['Circle Level Bonus'] = Circle_Bonus
Summary_Results['Circle Level Penalty'] = Circle_Penalty
Summary_Results['Summary Level Score'] = Summary_Score

Summary_Results['Final Score'] = Final_Score

In [26]:
Summary_Results_DF = pd.DataFrame.from_dict(data=Summary_Results, orient='index', columns=['Result'])



https://towardsdatascience.com/5-pandas-group-by-tricks-you-should-know-in-python-f53246c92c94

In [27]:
#Summary_Results_DF.columns

In [28]:
Summary_Results_DF.index = Summary_Results_DF.index.set_names('Category')
 
Summary_Results_DF.reset_index().head()

,Category,Result
0,Mentee Assigned More Than Once,First_5.Last_5@uvw.com
1,Uneven Circle Size,Yes
2,Span Circle Size,1 - 10
3,Uneven Circle Size Penalty,-2
4,Mentor assigned to 2 circles penalty,-2


In [29]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_multiple.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
Summary_Results_DF.to_excel(writer, sheet_name='highlevelsummary', header='False')
Circle_Summary_DF.to_excel(writer, sheet_name='circlesummary')
Result_DF.to_excel(writer, sheet_name='detail')

writer.close()

Group by Circle and Deprtment to check if not more than 2 mentees of same department are not in same circle